In [6]:
import pandas as pd

In [7]:

ruta = "INVENTARIO/inventario7sept.xlsx"
df = pd.read_excel(ruta)



FileNotFoundError: [Errno 2] No such file or directory: 'INVENTARIO/inventario7sept.xlsx'

In [ ]:
df.info()

In [ ]:
import pandas as pd
import re

def limpiar_y_transformar(df):
    
    # Crea una copia explícita del DataFrame para evitar el SettingWithCopyWarning
    df_limpio = df.copy()

    # 1. Clasificación categoría
    marcas_filtros = ["BALDWIN", "DONSSON", "AUT*PARTS", "RAMA", "RACOR USA", "RACOR BRASIL"]
    def clasificar_categoria(marca):
        marca = str(marca).upper().strip()
        if marca == "OTROS":
            return "Otros"
        elif marca in marcas_filtros:
            return "Filtro"
        else:
            return "Aceite/Lubricante"
    df_limpio["categoria_producto"] = df_limpio["Marca"].apply(clasificar_categoria)

    # 4. Extraer código interno
    def extraer_codigo(complete_name, categoria):
        if not isinstance(complete_name, str):
            return None
        if categoria == "Aceite/Lubricante":
            match = re.search(r"\[([A-Z0-9\-]+)\]", complete_name)
            if match:
                return match.group(1)
        elif categoria == "Filtro" or categoria == "Otros":
            match = re.search(r"\[([A-Z0-9\-]+)\]\s+(\S+)", complete_name)
            if match:
                codigo_base = match.group(1)
                palabra_siguiente = match.group(2)
                ultimos_digitos = ''.join(re.findall(r'\d', codigo_base))[-3:]
                return f"{ultimos_digitos} {palabra_siguiente}"
        return None
    df_limpio["Codigo_interno"] = df_limpio.apply(
        lambda row: extraer_codigo(row["Complete name"], row["categoria_producto"]),
        axis=1
    )

    # --- Lógica corregida para limpiar y transformar la columna 'Bodega' ---
    # Asume que ya tienes una columna 'warehouse_name' o 'Bodega' con el formato 'BD MP / Existencias'
    df_limpio['Sucursal_Estado_temp'] = df_limpio['Bodega'].str.split('/', n=1, expand=True)[1].str.strip()
    df_limpio['Sucursal'] = df_limpio['Sucursal_Estado_temp'].str.rsplit('/', n=1, expand=True)[0].str.strip()
    df_limpio['Estado'] = df_limpio['Sucursal_Estado_temp'].str.rsplit('/', n=1, expand=True)[1].str.strip()
    df_limpio = df_limpio.drop(columns=['Sucursal_Estado_temp'])

    mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }

    df_limpio['Sucursal'] =df_limpio['Sucursal'].map(mapa_sucursales)








    # 5. Eliminar productos con clase "SIN CLASE X SUCURSAL" y sin marca
    df_limpio = df_limpio[~(df_limpio['Marca'].isna() | (df_limpio['Marca'].str.strip() == ''))]
    df_limpio = df_limpio[df_limpio["Clase"] != "SIN CLASE X SUCURSAL"]
    
    # Seleccionamos las columnas finales en el orden deseado
    df_final = df_limpio[['Sucursal','Estado','Codigo_interno','Complete name','Clase','Marca','categoria_producto','Cantidad','Costo unitario','Costo total']]

    return df_final

In [ ]:
from datetime import datetime

df_final= limpiar_y_transformar(df)

fecha_match = re.search(r"\d{1,2}[a-zA-Z]{3}", ruta)
if fecha_match:
    fecha_inventario = fecha_match.group()  # "11ago"
else:
    # Si no se encuentra, pedirla manualmente
    fecha_inventario = input("Fecha del inventario (ej: 11ago): ")

# Guardar archivo limpio
nombre_archivo = f"inventario_limpio.xlsx"
df_final.to_excel(nombre_archivo, index=False)
